In [4]:
#!pip install safetensors>=0.4.1
#!pip install accelerate
#!pip install transformers -U

Found existing installation: torch 2.2.1

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Jensen R\anaconda3\Lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "C:\Users\Jensen R\anaconda3\Lib\site-packages\pip\_internal\commands\uninstall.py", line 110, in run
    uninstall_pathset.commit()
  File "C:\Users\Jensen R\anaconda3\Lib\site-packages\pip\_internal\req\req_uninstall.py", line 432, in commit
    self._moved_paths.commit()
  File "C:\Users\Jensen R\anaconda3\Lib\site-packages\pip\_internal\req\req_uninstall.py", line 278, in commit
    save_dir.cleanup()
  File "C:\Users\Jensen R\anaconda3\Lib\site-packages\pip\_internal\utils\temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "C:\Users\Jensen R\anaconda3\Lib\site-packages\pip\_vendor\tenacity\__init__.py", line 291, in wrapped_f
    return self(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Jensen R\anaconda3\Lib\si


Uninstalling torch-2.2.1:
  Successfully uninstalled torch-2.2.1


In [1]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import requests
from PIL import Image
import pandas as pd
from tqdm import tqdm

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", device_map="auto",offload_folder='model_offload')

# Read the CSV file
csv_file = 'sampled_products_5_each_img_front.csv'  # Adjust to your file path
df = pd.read_csv(csv_file)

# Column name containing the URLs
url_column = 'ItemDocumentValue'

# List to store generated captions
captions = []

# Process each image URL with a progress bar
for url in tqdm(df[url_column], desc="Processing images"):
    try:
        # Load and process the image
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()  # Ensure the request was successful
        image = Image.open(response.raw).convert('RGB')
        
        
        question = "Generate a thorough product description"
        inputs = processor(image, question, return_tensors="pt").to("cuda")
        
        out = model.generate(**inputs,max_new_tokens=100)
        generated_caption = processor.decode(out[0], skip_special_tokens=True).strip()
        
        # Append the caption to the list
        captions.append(generated_caption)
    except Exception as e:
        # Handle any exceptions
        captions.append(f"Error processing image: {e}")
        
# Create a new DataFrame with only the desired columns and captions
output_df = pd.DataFrame({
    'ItemSku': df['ItemSku'],
    'ItemDocumentValue': df['ItemDocumentValue'],
    'generated_captions': captions
})

# Save the new DataFrame to a CSV file
output_df.to_csv('BLIF2_output40.csv', index=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing images:  88%|███████████████████████████████████████████████████████▏       | 35/40 [11:09<00:52, 10.54s/it]C:\Users\Jensen R\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing images: 100%|███████████████████████████████████████████████████████████████| 40/40 [12:22<00:00, 18.57s/it]
